In [22]:
tickers = ['AAPL','JPM','F','PG', 'BA']

investment_period = 1 # each 1 investment days 
reward_period = 15 # each 15 investment periods
risk_aversions = [0,0.25,0.5,0.75,1]

In [ ]:
from enviroment import create_env
envs = [create_env(tickers,'1992-01-01','2018-18-31',investment_period,reward_period,risk_aversion) for risk_aversion in risk_aversions]

FileNotFoundError: [Errno 2] No such file or directory: './data/processed/P.csv'

In [ ]:
from stable_baselines3 import  PPO, SAC, DDPG
from stable_baselines3.common.base_class import BaseAlgorithm
learning_rate=0.001

algorithms = {
    "PPO" : PPO,
    "SAC" : SAC,
    "DDPG" : DDPG
}

models:list[BaseAlgorithm] = []

for algorithm in algorithms:
    for env in envs:
        model = algorithms[algorithm](policy='MlpPolicy', env=env, verbose=1, learning_rate=learning_rate)
        models.append(model)

Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device
Using cpu device


In [ ]:
for idx,model in enumerate(models):
    model.learn(total_timesteps=20000,log_interval=1000)
    risk_aversion = risk_aversions[idx%len(risk_aversions)]
    save_file_name = model.__class__.__name__ + f'_risk_{risk_aversion}_inv_{investment_period}_rew_{reward_period}.zip'
    model.save(f'./models/{save_file_name}')